# 문11) 해시법(오픈해시법, 체인법) 이란?

해시값 hash value, digest 

    기능 : 해시값을 인덱스로 하는 hash table에 접근할 수 있다.
    
    계산 :해시값은 해시 함수로 구한다.
        
        hash value = key % capacity
        
            key : 유저가 입력할 키 = 해시 테이블(배열)에 입력될 원소의 값
            capacity : hash table을 만든 사람이 설정한 해시테이블의 크기, 대개 prime number 사용
            key % capacity : hash function의 한 종류
            
    버킷 : 키와 밸류, 다음 버킷(노드)의 참조 정보를 가진 인스턴스(연결리스트의 노드)
        
        즉 버킷들은 자신의 해시값을 인덱스로 하는 해시테이블에 저장된다.
        
    
    작동구조 : 이제 사용자가 임의의 키를 입력하면 
             해시함수로 해당 키를 해시값으로 바꿔서, 
             해당 해시값을 인덱스로 하는 해시테이블의 버킷에 접근할 수 있다. 즉 검색할 수 있다. 
             이런 방식으로 버킷 추가, 삭제도 가능
    
    해시 충돌 : 서로 다른 키값이 같은 해시값을 가지게 되는 경우
    
    해시충돌 해결법 : 체인법, 오픈주소법.

        체인법(오픈 해시법) : 해시값이 같은 버킷들은 연결리스트로 연결된다. 
                          각 버킷은 같은 해시값에 특정 규칙으로 연결된 노드들을 참조한다.
                          잘 구현하면 O(1), 최악의 경우(모든 해시 충돌이 발생한 경우): O(n)
                          
                          체인법을 쓰는 언어 : C++, 자바, 고

    서로 다른 키가 같은 해시값을 갖는 건 괜찮지만
    똑같은 키를 중복해서 해시값에 맵핑할 수 없다.
    어떤 밸류를 가져와야 할지 알 수 없으므로.

    키만 다르다면 서로 독립된 노드에 접근 가능하게 만들 수 있다. 노드마다 다른 키를 갖게 되므로

# 문12) 해시테이블에서 해시충돌가능성과 메모리의 트레이드 오프

해시테이블을 아주 크게 만들면, 해시충돌을 더 줄일 수 있지만 메모리 낭비  
해시함수가 해시테이블 크기보다 작거나 같은 해시값을 만들어내게 하는 게 좋다

# 문13) 해시함수에 입력되는 키가 int가 아닌 문자열인 경우

    1. 문자열을 바이트문자열로 바꾸고 str(key).encode() 
    2. sha256 알고리즘을 통해 해시값으로 바꾸고 hashlib.sha256(str(key).encode())
    3. 바꾼 해시값을 다시 16진수로 바꾸고hashlib.sha256(str(key).encode()).hexdigest()
    4. 바꾼 16진수를 다시 10진수 정수로 바꾼다. int(hashlib.sha256(str(key).encode()).hexdigest(), 16)
    5. 이 정수로 capacity를 나눈 나머지를 해시값으로 쓴다.

In [81]:
import hashlib 

key = "love"
print(str(key).encode())
print(hashlib.sha256(str(key).encode()).digest())
print(hashlib.sha256(str(key).encode()).hexdigest())
print(int(hashlib.sha256(str(key).encode()).hexdigest(), 16))
print(int(hashlib.sha256(str(key).encode()).hexdigest(), 16) % 13)

b'love'
b'hotj\x95\xb6\xf86\xd7\xd7\x05g\xc3\x02\xc3\xf9\xeb\xb5\xee\r\xef=\x12 \xee\x9dN\x9f4\xf5\xe11'
686f746a95b6f836d7d70567c302c3f9ebb5ee0def3d1220ee9d4e9f34f5e131
47237459752947480255640604573901605112751966017541630774679148972986760290609
12


# 문14) 체인법 구현

노드 클래스, 해시클래스, 해시함수, search, add, remove, dump

In [2]:
from __future__ import annotations
from typing import Any, Type
import hashlib

class Node:
    """해시를 구성하는 노드(버킷)"""

    def __init__(self, key: Any, value: Any, next: Node) -> None:
        """초기화"""
        self.key   = key    # 유저가 입력할 키 값
        self.value = value  # 해당 키에 저장하고 싶은 데이터
        self.next  = next   # 다음 노드를 참조

In [53]:
class ChainedHash:
    """체인법으로 해시 클래스 구현"""

    def __init__(self, capacity: int) -> None:
        """초기화"""
        self.capacity = capacity             # 해시 테이블을 만든 사람이 설정한 테이블의 크기
        self.table = [None] * self.capacity  # 해시 테이블(리스트)을 선언

    def hash_value(self, key: Any) -> int:
        """해시값을 구함"""
        if isinstance(key, int):
            return key % self.capacity       # hash value(digest)를 인덱스로 하여 hash table에 접근
        return(int(hashlib.sha256(str(key).encode()).hexdigest(), 16) % self.capacity)
    
    def search(self, key: Any) -> Any:
        """key를 입력하면 value를 리턴"""
        hash = self.hash_value(key)  # 키에 해당하는 hash value 계산
        p = self.table[hash]         # p는 해당 해시값의 노드(버킷)를 참조

        while p is not None:         # 해당 해시값에 연결된 모든 노드를 검색
            if p.key == key:
                 return p.value  # 검색 성공
            p = p.next           # 아니면 p는 다음 노드를 참조

        return None              # 검색 실패

    def add(self, key: Any, value: Any) -> bool:
        """추가하려는 key와 해당 키의 value 추가"""
        hash = self.hash_value(key)  # 입력한 키의 해시값
        p = self.table[hash]         # p는 해당 해시값의 노드를 참조 (해당 해시가 비었다면 p는 None 을 참조)

        while p is not None:         # 빈 해시라면 while문 패스 / 해시 충돌이라면 해당 해시값에 연결된 모든 노드들을 돌며 키가 다른지 확인
            if p.key == key:                                # 추가하려는 키가 현재 노드의 키와 같다면
                return False                                     # 삽입 실패
            p = p.next                                      # 키가 다르다면 p는 다음노드를 참조
            
        temp = Node(key, value, self.table[hash]) 
        # 빈 해시였다면 추가하려는 키와 밸류로 새 노드를 만든다. 
            # 이때 추가된 노드의 self.next는 None을 참조 (self.table[hash] 값은 해당 해시값의 해시테이블 위치가 비어있으므로 None이다)
        # 해시 충돌이라면 키 검사를 끝냈으므로 추가하려는 키와 밸류로 새 노드를 만든다. 
            # 이때 추가된 노드의 self.next는 해당 해시값에 저장돼 있는 맨 첫번째 노드를 참조 
                # (self.table[hash] 값은 해당 해시값의 해시테이블의 맨 첫번째 버킷을 참조하고 있으므로)
            
        self.table[hash] = temp      # 해시테이블에 해당 해시값이 새로 추가된 노드를 참조하게 만든다.
                                         # 따라서 새로 추가된 노드는 해당 해시값에 연결된 첫번째 노드가 되고, 
                                         # 기존 첫번째 노드는 새로 추가된 노드의 다음 노드가 된다
        return True                  # 삽입 성공


    def remove(self, key: Any) -> bool:
        """키가 key인 원소를 삭제"""
        hash = self.hash_value(key)  # 삭제할 키의 해시값
        p = self.table[hash]         # p는 해당 해시의 노드를 참조
        pp = None                    # 바로 앞 주목 노드

        while p is not None:         # 해당 해시값에 연결된 모든 노드를 검색
            if p.key == key:  # 삭제하려는 key와 같은 키의 노드를 발견하면 
                if pp is None:
                    self.table[hash] = p.next # 해당노드를 삭제
                else:
                    pp.next = p.next  # p의 다음 노드는 p의 다음 다음 노드를 참조. i.e. p의 다음노드를 삭제
                return True  # key 삭제 성공
            pp = p                   # 삭제하려는 key와 노드의 키가 다르면, pp는 p를 참조
            p = p.next       # p는 다음 노드를 참조
            
        return False         # 삭제 실패(key가 존재하지 않음)

    def dump(self) -> None:
        """해시 테이블을 덤프"""
        for i in range(self.capacity):
            p = self.table[i]
            print(i, end='')
            while p is not None:
                print(f'  → {p.key} ({p.value})', end='')  # 해시 테이블의 해당 해시값과, 연결된 연결된 모든 노드의 키, 밸류 출력
                p = p.next
            print()

In [54]:
hash = ChainedHash(13)

In [55]:
hash.capacity

13

In [56]:
hash.table

[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [57]:
print(hash.search(13))

None


In [58]:
hash.add(13, "창호")

True

In [59]:
hash.table[0].key

13

In [60]:
hash.table[0].value

'창호'

In [61]:
print(hash.table[0].next)

None


In [62]:
hash.dump()

0  → 13 (창호)
1
2
3
4
5
6
7
8
9
10
11
12


In [63]:
hash.add(13, "화명")  # 이미 있는 키로는 추가 불가

False

In [64]:
hash.add(0, "화명")   # 해시값 충돌시 체인법 작동

True

In [65]:
print(hash.table[0].key)

0


In [66]:
print(hash.table[0].next.key)

13


In [67]:
print(hash.table[0].next.next)

None


In [68]:
hash.dump()

0  → 0 (화명)  → 13 (창호)
1
2
3
4
5
6
7
8
9
10
11
12


In [69]:
hash.add(26, "하늘")

True

In [70]:
print(hash.table[0].key)

26


In [71]:
print(hash.table[0].next.key)

0


In [72]:
print(hash.table[0].next.next.key)

13


In [73]:
hash.dump()

0  → 26 (하늘)  → 0 (화명)  → 13 (창호)
1
2
3
4
5
6
7
8
9
10
11
12


In [74]:
hash.search(0)

'화명'

# 문15) 오픈주소법(닫힌 해시법) 이란? 
해시충돌이 일어날 경우  
재해시함수를 사용해서 빈 해시를 찾는 방법: 파이썬 딕셔너리는 오픈주소법으로 구현돼 있다  
파이썬 딕셔너리가 오픈어드레싱을 쓰는 이유는 체이닝을 위한 연결리스트를 만드려면 추가 메모리 할당이 필요한데  
이 작업이 느린 작업이기 때문이다.  
    
    체인법(오픈해시법)은 사실상 무한정 저장이 가능하나
    오픈 어드레싱 방법을 쓰면 전체 해시테이블 크기가 저장가능 최대치다. 
    
        따라서 일정 이상 채워지면, 로드팩터가 일정값을 넘어서면 (파알 11장 해쉬테이블 로드팩터 참고)
        동적배열처럼 테이블 크기를 늘려줘야 한다.
        늘려주지 않으면, 탐사에 점점 더 오랜 시간이 걸린다.
        파이썬의 로드팩터는 0.66 (80%가 넘어가면 성능이 급격히 나빠진다)

재해시함수는 정하기 나름  
아래 방법은 오픈어드레싱 방법중 선형탐사법

    ex) 원래 해시함수가 key % capacity 라면
        재해시함수는 (기존해시함수값+1) % capacity 를 사용해서 해시충돌이 안날 때까지 키에 반복 적용
        예를 들어 capa가 13, 키가 18이었고 해시충돌이 일어 났다면
        그 다음엔 5+1 % capacity, 또 충돌나면 그 다음엔 (5+1 % capacity)+1 % capacity...
        
선형탐사법은 구현이 간단하고 성능이 좋지만 문제가 있다.  
클러스터링 : 해시테이블에 저장되는 데이터들이 고르게 분포되지 않고 뭉치는 경향  
해시테이블 여기 저기에 연속된 데이터 그룹(클러스터)이 생긴다.  
이렇게 되면 해시테이블 특정 위치들에 데이터가 몰리고, 다른 위치에는 상대적으로 데이터가 거의 없는 상태가 될 수 있다.


# 문16) 오픈주소법을 쓸 때 원소 검색, 추가, 삭제 시 특징

    재해시 함수를 사용하면, 
    키값의 원래해시함수 값과 다른 해시값 위치에 원소가 추가될 수 있다. (교재 145p 참고)

    이 경우 검색, 삭제를 할 때
    키값의 원래 해시함수 값의 버킷이 비어있더라도, 다른 해시값 위치에 검색 또는 삭제하고자 하는 키가 위치할 수 있다.
    
    따라서 오픈주소법에서는
    버킷의 상태를 알려주는 속성을 추가한다.
    EMPTY, OCCUPIED, DELETED
    
    검색시 해당 키의 해시 위치의 버킷이 EMPTY 라면 검색 중단
    OCCUPIED 라면 해당 해시 위치 버킷의 값 반환
    DELETED 라면 재해시해서 다른 해시 위치에 있는 해당 키가 있는지 반복문으로 검색한다.
    
    추가시 해당 키의 해시 위치의 버킷이 EMPTY 또는 DELETED 일 때 새 노드를 해당 해시값이 참조하게 한다.
    OCCUPIED 면 재해시
    
    삭제시 해당 키의 해시위치의 버킷 상태를 그냥 DELETED로만 바꿔놓으면 된다.
    


# 문17) 오픈주소법 구현

상태 클래스, 버킷 클래스, 해시클래스, 해시함수, 재해시함수, search_node, search, add, remove, dump

In [1]:
from __future__ import annotations
from typing import Any, Type
from enum import Enum
import hashlib

# 버킷의 속성
class Status(Enum):
    OCCUPIED = 0  # 데이터를 저장
    EMPTY = 1     # 비어 있음
    DELETED = 2   # 삭제 완료

class Bucket:
    """해시를 구성하는 버킷"""

    def __init__(self, key: Any = None, value: Any = None,
                       stat: Status = Status.EMPTY) -> None:
        """초기화"""
        self.key = key      # 키
        self.value = value  # 값
        self.stat = stat    # 속성

    def set(self, key: Any, value: Any, stat: Status) -> None:
        """모든 필드에 값을 설정"""
        self.key = key      # 키
        self.value = value  # 값
        self.stat = stat    # 속성

    def set_status(self, stat: Status) -> None:
        """속성을 설정"""
        self.stat = stat

class ClosedHash:
    """오픈 주소법을 구현하는 해시 클래스"""

    def __init__(self, capacity: int) -> None:
        """초기화"""
        self.capacity = capacity                 # 해시 테이블의 크기를 지정
        self.table = [Bucket()] * self.capacity  # 해시 테이블

    def hash_value(self, key: Any) -> int:
        """해시값을 구함"""
        if isinstance(key, int):
            return key % self.capacity
        return(int(hashlib.md5(str(key).encode()).hexdigest(), 16)
                % self.capacity)

    def rehash_value(self, key: Any) -> int:
        """재해시값을 구함"""
        return(self.hash_value(key) + 1) % self.capacity

    def search_node(self, key: Any) -> Any:
        """키가 key인 버킷을 검색"""
        hash = self.hash_value(key)  # 검색하는 키의 해시값
        p = self.table[hash]         # 버킷을 주목

        for _ in range(self.capacity):
            if p.stat == Status.EMPTY:
                break
            elif p.stat == Status.OCCUPIED and p.key == key:
                return p
            hash = self.rehash_value(hash)  # 재해시
            p = self.table[hash]
        return None

    def search(self, key: Any) -> Any:
        """키가 key인 갖는 원소를 검색하여 값을 반환"""
        p = self.search_node(key)
        if p is not None:
            return p.value  # 검색 성공
        else:
            return None     # 검색 실패

    def add(self, key: Any, value: Any) -> bool:
        """키가 key이고 값이 value인 요소를 추가"""
        if self.search(key) is not None:
            return False             # 이미 등록된 키

        hash = self.hash_value(key)  # 추가하는 키의 해시값
        p = self.table[hash]         # 버킷을 주목
        for _ in range(self.capacity):
            if p.stat == Status.EMPTY or p.stat == Status.DELETED:
                self.table[hash] = Bucket(key, value, Status.OCCUPIED)
                return True
            hash = self.rehash_value(hash)  # 재해시
            p = self.table[hash]
        return False                        # 해시 테이블이 가득 참

    def remove(self, key: Any) -> int:
        """키가 key인 갖는 요소를 삭제"""
        p = self.search_node(key)  # 버킷을 주목
        if p is None:
            return False           # 이 키는 등록되어 있지 않음
        p.set_status(Status.DELETED)
        return True

    def dump(self) -> None:
        """해시 테이블을 덤프"""
        for i in range(self.capacity):
            print(f'{i:2} ', end='')
            if self.table[i].stat == Status.OCCUPIED:
                print(f'{self.table[i].key} ({self.table[i].value})')
            elif self.table[i].stat == Status.EMPTY:
                print('-- 미등록 --')
            elif self.table[i] .stat == Status.DELETED:
                print('-- 삭제 완료 --')